In [0]:
!pip install pyshp

In [0]:
import shapefile
#Läser in datamängd AK. 
#Dessa behöver tankas ner från lantmäteriet Sverigekartor 1:1 miljon. 
#Filer finns i sve_1milj_Sweref_99_TM_shape/svk/riks/ak_riks.{cpg|dbf|prj|shp|shx}
#Ladda upp samtliga till colab.
sf = shapefile.Reader("ak_riks", encoding="latin1")
len(sf.records()) #295 kommuner


Koden nedan hämtades från [https://gis.stackexchange.com/questions/292637/from-sweref99-to-epsg3857](https://gis.stackexchange.com/questions/292637/from-sweref99-to-epsg3857). Datan från lantmäteriet är inte lat/lng utan ett annat koordinatsystem och behöver omvandlas.

In [0]:
#@title Omvandling grid till lat/lng
import math

axis = None # Semi-major axis of the ellipsoid.
flattening = None # Flattening of the ellipsoid.
central_meridian = None # Central meridian for the projection.
lat_of_origin = None # Latitude of origin.
scale = None # Scale on central meridian.
false_northing = None # Offset for origo.
false_easting = None # Offset for origo.

# Parameters for SWEREF99TM.
# Parameter: projection (string). Must match if-statement.
def swedish_params(projection) :

    global central_meridian 
    global scale 
    global false_northing 
    global false_easting 
    global lat_of_origin
    # SWEREF99TM and SWEREF99ddmm  parameters.
    if (projection == "sweref_99_tm") :
        sweref99_params()
        central_meridian = 15.00
        lat_of_origin = 0.0
        scale = 0.9996
        false_northing = 0.0
        false_easting = 500000.0

def sweref99_params() :

    global axis 
    global flattening 
    global central_meridian 
    global lat_of_origin 
    global scale 
    global false_northing 
    global false_easting 

    axis = 6378137.0 # GRS 80.
    flattening = 1.0 / 298.257222101 # GRS 80.
    central_meridian = None
    lat_of_origin = 0.0
    scale = 1.0
    false_northing = 0.0
    false_easting = 150000.0

# Conversion from grid coordinates to geodetic coordinates.
def grid_to_geodetic(x, y) :
    lat_lon = [0] * 2
    if (central_meridian == None) :
        return lat_lon

    # Prepare ellipsoid-based stuff.
    e2 = flattening * (2.0 - flattening)
    n = flattening / (2.0 - flattening)
    a_roof = axis / (1.0 + n) * (1.0 + n*n/4.0 + n*n*n*n/64.0)
    delta1 = n/2.0 - 2.0*n*n/3.0 + 37.0*n*n*n/96.0 - n*n*n*n/360.0
    delta2 = n*n/48.0 + n*n*n/15.0 - 437.0*n*n*n*n/1440.0
    delta3 = 17.0*n*n*n/480.0 - 37*n*n*n*n/840.0
    delta4 = 4397.0*n*n*n*n/161280.0

    Astar = e2 + e2*e2 + e2*e2*e2 + e2*e2*e2*e2
    Bstar = -(7.0*e2*e2 + 17.0*e2*e2*e2 + 30.0*e2*e2*e2*e2) / 6.0
    Cstar = (224.0*e2*e2*e2 + 889.0*e2*e2*e2*e2) / 120.0
    Dstar = -(4279.0*e2*e2*e2*e2) / 1260.0

    # Convert.
    deg_to_rad = math.pi / 180
    lambda_zero = central_meridian * deg_to_rad
    xi = (x - false_northing) / (scale * a_roof)       
    eta = (y - false_easting) / (scale * a_roof)
    xi_prim = xi - delta1*math.sin(2.0*xi) * math_cosh(2.0*eta) - delta2*math.sin(4.0*xi) * math_cosh(4.0*eta) - delta3*math.sin(6.0*xi) * math_cosh(6.0*eta) - delta4*math.sin(8.0*xi) * math_cosh(8.0*eta)
    eta_prim = eta - delta1*math.cos(2.0*xi) * math_sinh(2.0*eta) - delta2*math.cos(4.0*xi) * math_sinh(4.0*eta) - delta3*math.cos(6.0*xi) * math_sinh(6.0*eta) - delta4*math.cos(8.0*xi) * math_sinh(8.0*eta)
    phi_star = math.asin(math.sin(xi_prim) / math_cosh(eta_prim))
    delta_lambda = math.atan(math_sinh(eta_prim) / math.cos(xi_prim))
    lon_radian = lambda_zero + delta_lambda
    lat_radian = phi_star + math.sin(phi_star) * math.cos(phi_star) * (Astar + Bstar*math.pow(math.sin(phi_star), 2) + Cstar*math.pow(math.sin(phi_star), 4) + Dstar*math.pow(math.sin(phi_star), 6))    
    lat_lon[0] = lat_radian * 180.0 / math.pi
    lat_lon[1] = lon_radian * 180.0 / math.pi
    return lat_lon


# Missing defs in the math library.
def math_sinh(value) :
    return 0.5 * (math.exp(value) - math.exp(-value))

def math_cosh(value) :
    return 0.5 * (math.exp(value) + math.exp(-value))

def math_atanh(value) :
    return 0.5 * math.log((1.0 + value) / (1.0 - value))

swedish_params("sweref_99_tm")

In [0]:
!pip install geojson
from geojson import Point, Feature, FeatureCollection, dump

#Exempel omvandla en lista med punkter till lat/lng
xy = sf.shape(0).points
latlng = [tuple(grid_to_geodetic(y,x)) for (x,y) in xy]

#Omvandla hela datamängden till lat/lng
geodata = sf.__geo_interface__
#print(geodata['features'][0])
for feature in geodata['features']:
  coords = feature['geometry']['coordinates']
  coords = list(coords)
  coords[0] = list(coords[0])
  for i,(x,y) in enumerate(coords[0]):
    #GeoJSON ska sparas i lng/lat istället för lat/lng
    coords[0][i] = grid_to_geodetic(y,x)[::-1]
  feature['geometry']['coordinates'] = coords

#Omvandla bounding box
geodata['bbox'] = geodata['bbox'].tolist()
bbx1,bby1,bbx2,bby2 = geodata['bbox']
bblat1,bblng1 = grid_to_geodetic(bby1,bbx1)
bblat2,bblng2 = grid_to_geodetic(bby2,bbx2)
geodata['bbox'] = [bblng1,bblat1,bblng2,bblat2]


with open('kommuner.geojson', 'w') as f:
   dump(geodata, f)